In [1]:
#!pip install spacy-langdetect
#!pip install language-detector
#!pip install symspellpy
#!pip install sentence-transformers

In [2]:
# import os
# import json
# from tqdm import tqdm

# import time
# from sklearn.decomposition import PCA
# from sklearn.manifold import TSNE
# import seaborn as sns
# from mpl_toolkits.mplot3d import Axes3D

# 1. Read dataset.

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np

In [4]:
meta = pd.read_csv("../metadata.csv")
print(meta.shape)

(341713, 19)


C:\Users\zhipe\Anaconda3\envs\tm\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
meta.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN


Filter meta file by selecting only papers after 2020.

In [6]:
meta["publish_time"] = pd.to_datetime(meta["publish_time"])
meta["publish_year"] = (pd.DatetimeIndex(meta['publish_time']).year)
meta["publish_month"] = (pd.DatetimeIndex(meta['publish_time']).month)
meta = meta[meta["publish_year"] == 2020]
print(meta.shape[0], " papers are available after 2020 Jan 1.")

247255  papers are available after 2020 Jan 1.


In [7]:
#meta.groupby(["publish_month","publish_year"]).size().plot(kind='bar')
meta.groupby(["publish_month","publish_year"]).size()
#tmp = meta.groupby(["journal"])["journal"].count()
#tmp=tmp.sort_index()
#print(tmp.size)

publish_month  publish_year
1.0            2020.0          123040
2.0            2020.0            1881
3.0            2020.0            4861
4.0            2020.0           12299
5.0            2020.0           18171
6.0            2020.0           17611
7.0            2020.0           17723
8.0            2020.0           16545
9.0            2020.0           17559
10.0           2020.0           14888
11.0           2020.0            2280
12.0           2020.0             397
dtype: int64

In [8]:
print(meta.columns)

Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id', 'publish_year', 'publish_month'],
      dtype='object')


Count how many documents has abstract

In [9]:
count = 0
index = []
for i in range(len(meta)):
    #print(i)
    if type(meta.iloc[i, 8])== float:
        count += 1
    else:
        index.append(i)

print(len(index), " papers have abstract available.")

163820  papers have abstract available.


Create dataframe with all abstracts and use it as input corpus.

In [10]:
documents = meta.iloc[index, 8]
documents=documents.reset_index()
documents.head()
documents.drop("index", inplace = True, axis = 1)
documents["index"] = documents.index.values
documents.head(3)

,abstract,index
0,BACKGROUND: Dexmedetomidine has been reported ...,0
1,BACKGROUND: Global end-diastolic volume (GEDV)...,1
2,BACKGROUND: Human metapneumovirus (HMPV) is an...,2


# 2. Preprocessing functions

In [11]:
from symspellpy import SymSpell, Verbosity
import pkg_resources

sym_spell = SymSpell(max_dictionary_edit_distance=3, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
if sym_spell.word_count:
    pass
else:
    sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

print(sym_spell.word_count)

82781


Sentence level preprocess
* Lowercase + base filter
* Some basic normalization

In [12]:
import re
def f_base(s):
    """
    :param s: string to be processed
    :return: processed string: see comments in the source code for more info
    """
    # normalization 1: xxxThis is a --> xxx. This is a (missing delimiter)
    s = re.sub(r'([a-z])([A-Z])', r'\1\. \2', s)  # before lower case
    # normalization 2: lower case
    s = s.lower()
    # normalization 3: "&gt", "&lt"
    s = re.sub(r'&gt|&lt', ' ', s)
    # normalization 4: letter repetition (if more than 2)
    s = re.sub(r'([a-z])\1{2,}', r'\1', s)
    # normalization 5: non-word repetition (if more than 1)
    s = re.sub(r'([\W+])\1{1,}', r'\1', s)
    # normalization 6: string * as delimiter
    s = re.sub(r'\*|\W\*|\*\W', '. ', s)
    # normalization 7: stuff in parenthesis, assumed to be less informal
    s = re.sub(r'\(.*?\)', '. ', s)
    # normalization 8: xxx[?!]. -- > xxx.
    s = re.sub(r'\W+?\.', '.', s)
    # normalization 9: [.?!] --> [.?!] xxx
    s = re.sub(r'(\.|\?|!)(\w)', r'\1 \2', s)
    # normalization 10: ' ing ', noise text
    s = re.sub(r' ing ', ' ', s)
    # normalization 11: noise text
    s = re.sub(r'product received for free[.| ]', ' ', s)
    # normalization 12: phrase repetition
    s = re.sub(r'(.{2,}?)\1{1,}', r'\1', s)
    return s.strip()

In [13]:
from language_detector import detect_language
# language detection
def f_lan(s):
    """
    :param s: string to be processed
    :return: boolean (s is English)
    """
    # some reviews are actually english but biased toward french
    return detect_language(s) in {'English', 'French','Spanish','Chinese'}

 Word level preprocess

In [14]:
import nltk
# from nltk.corpus import wordnet
# from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
# filtering out punctuations and numbers
def f_punct(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with punct and number filter out
    """
    return [word for word in w_list if word.isalpha()]

# selecting nouns
def f_noun(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with only nouns selected
    """
    return [word for (word, pos) in nltk.pos_tag(w_list) if pos[:2] == 'NN']

# typo correction
def f_typo(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with typo fixed by symspell. words with no match up will be dropped
    """
    w_list_fixed = []
    for word in w_list:
        suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=3)
        # print(suggestions[0])
        if suggestions:
            w_list_fixed.append(suggestions[0].term)
        else:
            w_list_fixed.append(word)
    return w_list_fixed

# stemming if doing word-wise
p_stemmer = PorterStemmer()
def f_stem(w_list):
    """
    :param w_list: word list to be processed
    :return: w_list with stemming
    """
    return [p_stemmer.stem(word) for word in w_list]

In [15]:
# from nltk.corpus import stopwords
# filtering out stop words
# create English stop words list
from stop_words import get_stop_words
stop_words = (list(
    set(get_stop_words('en'))
    |set(get_stop_words('es'))
    |set(get_stop_words('de'))
    |set(get_stop_words('it'))
    |set(get_stop_words('ca'))
    |set(get_stop_words('pt'))
    |set(get_stop_words('pl'))
    |set(get_stop_words('da'))
    |set(get_stop_words('ru'))
    |set(get_stop_words('sv'))
    |set(get_stop_words('sk'))
    |set(get_stop_words('nl'))
))

def f_stopw(w_list):
    """
    filtering out stop words
    """
    return [word for word in w_list if word not in stop_words]

In [16]:
def preprocess_sent(rw):
    """
    Get sentence level preprocessed data from raw review texts
    :param rw: review to be processed
    :return: sentence level pre-processed review
    """
    s = f_base(rw)
    if not f_lan(s):
        return None
    return s

from nltk.tokenize import word_tokenize
def preprocess_word(s):
    """
    Get word level preprocessed data from preprocessed sentences
    including: remove punctuation, select noun, fix typo, stem, stop_words
    :param s: sentence to be processed
    :return: word level pre-processed review
    """
    if not s:
        return None
    w_list = word_tokenize(s)
    w_list = f_punct(w_list)
    w_list = f_noun(w_list)
    w_list = f_typo(w_list)
    w_list = f_stem(w_list)
    w_list = f_stopw(w_list)

    return w_list

# 3. Preprocessing raw text

In [17]:
documents = documents.fillna('')  # only the comments has NaN's
docs = documents.abstract

In [18]:
samp_size = 5100 #51000
ntopic = 10
np.random.seed(100)

In [19]:
n_docs = len(docs)
sentences = []  # sentence level preprocessed
token_lists = []  # word level preprocessed
idx_in = []  # index of sample selected
samp = np.random.choice(n_docs, samp_size)

In [20]:
docs.head()

0    BACKGROUND: Dexmedetomidine has been reported ...
1    BACKGROUND: Global end-diastolic volume (GEDV)...
2    BACKGROUND: Human metapneumovirus (HMPV) is an...
3    BACKGROUND: Antimicrobial resistance (AMR) com...
4    Glycoconjugate vaccines based on bacterial cap...
Name: abstract, dtype: object

In [21]:
for i, idx in enumerate(samp):
    sentence = preprocess_sent(docs[idx])
    token_list = preprocess_word(sentence)
    if token_list:
        idx_in.append(idx)
        sentences.append(sentence)
        token_lists.append(token_list)
    print('{} %'.format(str(np.round((i + 1) / len(samp) * 100, 2))), end='\r')

In [22]:
print(len(sentences))
print(len(token_lists))


5062
5062


# 4. Define model object

In [23]:
import gensim
# from gensim.utils import simple_preprocess
from gensim import corpora, models
# from gensim.parsing.preprocessing import STOPWORDS
# import datetime
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

class Topic_Model:
    def __init__(self, k=10, method='TFIDF'):
        """
        :param k: number of topics
        :param method: method chosen for the topic model
        """
        if method not in {'TFIDF', 'LDA'}:
            raise Exception('Invalid method!')
        self.k = k
        self.dictionary = None
        self.corpus = None
        self.cluster_model = None
        self.ldamodel = None
        self.vec = {}
        self.gamma = 15  # parameter for relative importance of lda
        self.method = method
        self.AE = None
        self.id = method + '_' + datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

    def vectorize(self, sentences, token_lists, method=None):
        """
        Get vecotr representations from selected methods
        """
        # Default method
        if method is None:
            method = self.method

        # turn tokenized documents into a id <-> term dictionary
        self.dictionary = corpora.Dictionary(token_lists)
        # convert tokenized documents into a document-term matrix
        self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

        if method == 'TFIDF':
            print('Getting vector representations for TF-IDF ...')
            tfidf = TfidfVectorizer()
            vec = tfidf.fit_transform(sentences)
            print('Getting vector representations for TF-IDF. Done!')
            return vec

        elif method == 'LDA':
            print('Getting vector representations for LDA ...')
            if not self.ldamodel:
                self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, id2word=self.dictionary,
                                                                passes=20)

            def get_vec_lda(model, corpus, k):
                """
                Get the LDA vector representation (probabilistic topic assignments for all documents)
                :return: vec_lda with dimension: (n_doc * n_topic)
                """
                n_doc = len(corpus)
                vec_lda = np.zeros((n_doc, k))
                for i in range(n_doc):
                    # get the distribution for the i-th document in corpus
                    for topic, prob in model.get_document_topics(corpus[i]):
                        vec_lda[i, topic] = prob

                return vec_lda

            vec = get_vec_lda(self.ldamodel, self.corpus, self.k)
            print('Getting vector representations for LDA. Done!')
            return vec

    def fit(self, sentences, token_lists, method=None, m_clustering=None):
        """
        Fit the topic model for selected method given the preprocessed data
        :docs: list of documents, each doc is preprocessed as tokens
        :return:
        """
        # Default method
        if method is None:
            method = self.method
        # Default clustering method
        if m_clustering is None:
            m_clustering = KMeans

        # turn tokenized documents into a id <-> term dictionary
        if not self.dictionary:
            self.dictionary = corpora.Dictionary(token_lists)
            # convert tokenized documents into a document-term matrix
            self.corpus = [self.dictionary.doc2bow(text) for text in token_lists]

        ####################################################
        #### Getting ldamodel or vector representations ####
        ####################################################

        if method == 'LDA':
            if not self.ldamodel:
                print('Fitting LDA ...')
                self.ldamodel = gensim.models.ldamodel.LdaModel(self.corpus, num_topics=self.k, id2word=self.dictionary,
                                                                passes=20)
                print('Fitting LDA Done!')
        else:
            print('Clustering embeddings ...')
            self.cluster_model = m_clustering(self.k)
            self.vec[method] = self.vectorize(sentences, token_lists, method)
            self.cluster_model.fit(self.vec[method])
            print('Clustering embeddings. Done!')

    def predict(self, sentences, token_lists, out_of_sample=None):
        """
        Predict topics for new_documents
        """
        # Default as False
        out_of_sample = out_of_sample is not None

        if out_of_sample:
            corpus = [self.dictionary.doc2bow(text) for text in token_lists]
            if self.method != 'LDA':
                vec = self.vectorize(sentences, token_lists)
                print(vec)
        else:
            corpus = self.corpus
            vec = self.vec.get(self.method, None)

        if self.method == "LDA":
            lbs = np.array(list(map(lambda x: sorted(self.ldamodel.get_document_topics(x),
                                                     key=lambda x: x[1], reverse=True)[0][0],
                                    corpus)))
        else:
            lbs = self.cluster_model.predict(vec)
        return lbs

In [24]:
import pickle
import warnings
warnings.filterwarnings('ignore', category=Warning)

method = "LDA" #"LDA_BERT"
# Define the topic model object
tm = Topic_Model(k = ntopic, method = method)
# Fit the topic model by chosen method
tm.fit(sentences, token_lists)

Fitting LDA ...
Fitting LDA Done!


In [25]:
# Evaluate using metrics
with open("{}.file".format(tm.id), "wb") as f:
    pickle.dump(tm, f, pickle.HIGHEST_PROTOCOL)

In [28]:
from collections import Counter
from sklearn.metrics import silhouette_score
import umap
from wordcloud import WordCloud
from gensim.models.coherencemodel import CoherenceModel
# import numpy as np
import os

Coherence: 0.4644990408565503
Silhouette Score: None


In [32]:
cm = CoherenceModel(model=tm.ldamodel, texts=token_lists, corpus=tm.corpus,
                    dictionary=tm.dictionary, coherence="c_v")
print(cm.get_coherence())
print('Coherence:', cm.get_coherence())

0.4644990408565503


In [35]:
for topic in tm.ldamodel.print_topics():
    print(topic)

(0, '0.033*"studi" + 0.019*"result" + 0.016*"particip" + 0.015*"survey" + 0.015*"student" + 0.013*"anxieti" + 0.013*"health" + 0.011*"level" + 0.011*"effect" + 0.010*"method"')
(1, '0.037*"test" + 0.030*"method" + 0.026*"sampl" + 0.025*"detect" + 0.022*"result" + 0.018*"sensit" + 0.015*"assay" + 0.013*"perform" + 0.012*"valu" + 0.011*"system"')
(2, '0.025*"infect" + 0.024*"drug" + 0.023*"viru" + 0.022*"vaccin" + 0.020*"coronaviru" + 0.018*"protein" + 0.017*"diseas" + 0.013*"respiratori" + 0.011*"cell" + 0.010*"studi"')
(3, '0.094*"patient" + 0.024*"diseas" + 0.018*"studi" + 0.017*"treatment" + 0.016*"infect" + 0.016*"case" + 0.015*"risk" + 0.013*"result" + 0.013*"hospit" + 0.013*"group"')
(4, '0.057*"cancer" + 0.011*"tutor" + 0.009*"water" + 0.009*"energi" + 0.008*"studi" + 0.007*"result" + 0.007*"analysi" + 0.007*"seed" + 0.006*"eye" + 0.006*"effect"')
(5, '0.046*"cell" + 0.015*"express" + 0.013*"activ" + 0.013*"effect" + 0.013*"group" + 0.012*"cytokin" + 0.012*"level" + 0.011*"lung" 

# 5. Visualization

Visualization functions

In [ ]:
def plot_proj(embedding, lbs):
    """
    Plot UMAP embeddings
    :param embedding: UMAP (or other) embeddings
    :param lbs: labels
    """
    n = len(embedding)
    counter = Counter(lbs)
    for i in range(len(np.unique(lbs))):
        plt.plot(embedding[:, 0][lbs == i], embedding[:, 1][lbs == i], '.', alpha=0.5,
                 label='cluster {}: {:.2f}%'.format(i, counter[i] / n * 100))
    plt.legend(loc = 'best')
    plt.grid(color ='grey', linestyle='-',linewidth = 0.25)

def visualize(model):
    """
    Visualize the result for the topic model by 2D embedding (UMAP)
    :param model: Topic_Model object
    """
    if model.method == 'LDA':
        return
    reducer = umap.UMAP()
    print('Calculating UMAP projection ...')
    vec_umap = reducer.fit_transform(model.vec[model.method])
    print('Calculating UMAP projection. Done!')
    plot_proj(vec_umap, model.cluster_model.labels_)
    dr = '/kaggle/working/contextual_topic_identification/docs/images/{}/{}'.format(model.method, model.id)
    if not os.path.exists(dr):
        os.makedirs(dr)
    plt.savefig('/kaggle/working/2D_vis')

def get_wordcloud(model, token_lists, topic):
    """
    Get word cloud of each topic from fitted model
    :param model: Topic_Model object
    :param sentences: preprocessed sentences from docs
    """
    if model.method == 'LDA':
        return
    print('Getting wordcloud for topic {} ...'.format(topic))
    lbs = model.cluster_model.labels_
    tokens = ' '.join([' '.join(_) for _ in np.array(token_lists)[lbs == topic]])

    wordcloud = WordCloud(width=800, height=560,
                          background_color='white', collocations=False,
                          min_font_size=10).generate(tokens)

    # plot the WordCloud image
    plt.figure(figsize=(8, 5.6), facecolor=None)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.tight_layout(pad=0)
    dr = '/kaggle/working/{}/{}'.format(model.method, model.id)
    if not os.path.exists(dr):
        os.makedirs(dr)
    plt.savefig('/kaggle/working' + '/Topic' + str(topic) + '_wordcloud')
    print('Getting wordcloud for topic {}. Done!'.format(topic))

In [41]:
# visualize and save img
visualize(tm)
for i in range(tm.k):
    get_wordcloud(tm, token_lists, i)


Coherence: 0.4644990408565503
Silhouette Score: None
